In [2]:
from flask import Flask
from flask import request
import werkzeug
import cv2
from PIL import Image
import mediapipe as mp
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Class containing Final Results for MediaPipe

In [3]:
class Results:
    
    cols = ["x","y","z","vis"]
    #TRAGULAR DATA
    trag_cols = ["x","y","z","vis"]
    trag_left_rows = ["left_ear","left_index","left_elbow","left_wrist"]
    trag_right_rows = ["right_ear","right_index","right_elbow","right_wrist"]
    tragular_left_coord = []
    tragular_right_coord = []
    
    #LUMBAR DATA
    lumbar_cols = ["x","y","z","vis"]
    lumbar_rows = ["left_index_world","right_index_world","left_index_hol","right_index_hol",
                  "left_wrist","right_wrist","left_elbow","right_elbow"]
    lumbar_neutral_coord = []
    lumbar_left_coord = []
    lumbar_right_coord = []
    
    #INTERMALLEOLAR DATA
    intermalleolar_rows = ["left_ankle_world","right_ankle_world","left_ankle_hol"
                           ,"right_ankle_hol","left_knee_hol","right_knee_hol"]
    intermalleolar_coord = []
    
    #CERVICAL DATA
    cervical_cols = ["x","y","z","vis"]
    cervical_left_coord = []
    cervical_neutral_coord = []
    cervical_right_coord = []
    
    #FLEXION DATA
    flexion_neutral_left_rows = ["left_shoulder_neutral_world","left_shoulder_neutral_norm",
                         "left_hip_neutral_world","left_hip_neutral_norm"]
    flexion_extension_left_rows = ["left_shoulder_extension_world","left_shoulder_extension_norm",
                         "left_hip_extension_world","left_hip_extension_norm"]
    flexion_neutral_right_rows = ["right_shoulder_neutral_world","right_shoulder_neutral_norm",
                         "right_hip_neutral_world","right_hip_neutral_norm"]
    flexion_extension_right_rows = ["right_shoulder_extension_world","right_shoulder_extension_norm",
                         "right_hip_extension_world","right_hip_extension_norm"]
    
    flexion_neutral_left_coord = []
    flexion_neutral_right_coord = []
    flexion_extension_left_coord = []
    flexion_extension_right_coord = []
    
    

In [4]:
def draw(annotated_image,results,mp_holistic):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    
    mp_drawing.draw_landmarks(
        annotated_image, 
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS)
    
    mp_drawing.draw_landmarks(
        annotated_image, 
        results.right_hand_landmarks, 
        mp_holistic.HAND_CONNECTIONS)
    
    mp_drawing.draw_landmarks(
        annotated_image,
        results.face_landmarks,
        mp_holistic.FACEMESH_TESSELATION,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.
        get_default_pose_landmarks_style())
    

In [5]:
#Function used to calculate relevant lumbar coordinates
def mp_pose_lumbar(side,file):
    
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("lumbar.png",image)

    coordinate_list = [[results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].x,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].y,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].z,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].visibility],
                           [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].x,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].y,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].z,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].visibility]]

    
    if(side == -1):
        Results.lumbar_left_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_left_coord.index = Results.lumbar_rows
        print(Results.lumbar_left_coord)
    elif(side == 0):
        Results.lumbar_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_neutral_coord.index = Results.lumbar_rows
        print(Results.lumbar_neutral_coord)
    else:
        Results.lumbar_right_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_right_coord.index = Results.lumbar_rows
        print(Results.lumbar_right_coord)
    
    

# Functions for Measurements
    

In [6]:
#Function used to calculate the relevant tragular coordinates
def mp_pose_tragular(side,file):
    mp_holistic = mp.solutions.holistic 

    holistic = mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        refine_face_landmarks=True)
    
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    
    draw(annotated_image,results,mp_holistic)
    
    if(side == 0):
        cv2.imwrite("left_tragular_annotated.png",annotated_image)
        left_hand = results.left_hand_landmarks
        
        left_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].visibility]
        
        left_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].visibility]
        
        left_ear_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].visibility]
        
        left_index_coord = [left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                       left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                       left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                       left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
        
        
        
        coordinate_list = [left_ear_coord,left_index_coord,left_elbow_coord,left_wrist_coord]
        
        Results.tragular_left_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_left_coord.index = Results.trag_left_rows
        print(Results.tragular_left_coord)
    else:
        cv2.imwrite("right_tragular_annotated.png",annotated_image)
        right_hand = results.right_hand_landmarks
        
        right_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].visibility]
        
        right_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].visibility]
        
        right_ear_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].visibility]
        
        right_index_coord = [right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                       right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                       right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                       right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
        
        
        coordinate_list = [right_ear_coord,right_index_coord,right_elbow_coord,right_wrist_coord]
        
        Results.tragular_right_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_right_coord.index = Results.trag_right_rows
        print(Results.tragular_right_coord)
    

In [7]:
def mp_lumbar_holistic(side, file):
    mp_holistic = mp.solutions.holistic 
    
    holistic = mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        refine_face_landmarks=True)
    
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    
    draw(annotated_image,results,mp_holistic)
    
    cv2.imwrite("lumbar.png",annotated_image)
    
    right_hand = results.right_hand_landmarks
    left_hand = results.left_hand_landmarks
    
    left_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].visibility]
        
    left_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].visibility]
        
    left_index_coord_hol = [left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                    left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                    left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                    left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
    
    left_index_coord_world = [results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].x,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].y,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].z,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].visibility]
        
    right_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].visibility]
        
    right_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].visibility]
        
    right_index_coord_hol = [right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                    right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                    right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                    right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
        
    right_index_coord_world =  [results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].x,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].y,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].z,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].visibility]

    coordinate_list = [left_index_coord_world, right_index_coord_world,  left_index_coord_hol, right_index_coord_hol,
                      left_wrist_coord, right_wrist_coord, left_elbow_coord, right_elbow_coord]

    
    if(side == -1):
        Results.lumbar_left_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_left_coord.index = Results.lumbar_rows
        print(Results.lumbar_left_coord)
        
    elif(side == 0):
        Results.lumbar_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_neutral_coord.index = Results.lumbar_rows
        print(Results.lumbar_neutral_coord)
        
    else:
        Results.lumbar_right_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_right_coord.index = Results.lumbar_rows
        print(Results.lumbar_right_coord)
    
    

In [8]:
def mp_pose_intermalleolar(file):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))

    cv2.imwrite("intermalleolar_annotated.png",image)
    
    left_ankle_world = results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
    right_ankle_world = results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
    left_ankle_norm = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
    right_ankle_norm = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
    left_knee_norm = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
    right_knee_norm = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]
    
    coordinate_list = [[left_ankle_world.x,
                        left_ankle_world.y,
                        left_ankle_world.z,
                        left_ankle_world.visibility],
                       
                        [right_ankle_world.x,
                         right_ankle_world.y,
                         right_ankle_world.z,
                         right_ankle_world.visibility],
                       
                        [left_ankle_norm.x,
                         left_ankle_norm.y,
                         left_ankle_norm.z,
                         left_ankle_norm.visibility],
                       
                        [right_ankle_norm.x,
                         right_ankle_norm.y,
                         right_ankle_norm.z,
                         right_ankle_norm.visibility],
                      
                        [left_knee_norm.x,
                         left_knee_norm.y,
                         left_knee_norm.z,
                         left_knee_norm.visibility],
                       
                        [right_knee_norm.x,
                         right_knee_norm.y,
                         right_knee_norm.z,
                         right_knee_norm.visibility]]
    
    Results.intermalleolar_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
    Results.intermalleolar_coord.index = Results.intermalleolar_rows
    

In [9]:
def mp_pose_cervical(file,side):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=50, circle_radius=10),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=50, circle_radius=10))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    
    
    nose_world = results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
    right_ear_world = results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR]
    left_ear_world = results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR]
    nose_norm = results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
    right_ear_norm = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR]
    left_ear_norm = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR]

    if side == -1:
        cv2.imwrite("left_cervical.png",image)
        coordinate_list = [[nose_world.x, nose_world.y, nose_world.z, nose_world.visibility],
                            
                           [right_ear_world.x, right_ear_world.y, right_ear_world.z, right_ear_world.visibility],
                            
                           [nose_norm.x, nose_norm.y, nose_norm.z, nose_norm.visibility],
                           
                           [right_ear_norm.x, right_ear_norm.y, right_ear_norm.z, right_ear_norm.visibility]]
            
        Results.cervical_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_left_coord.index = ["left_nose_world_coordinates", "right_ear_world_coordinates",
                                            "left_nose_norm_coordinates","right_ear_norm_coordinates"]
        print(Results.cervical_left_coord)
    
    elif side == 0:
        cv2.imwrite("neutral_cervical.png",image)
        coordinate_list = [[nose_world.x, nose_world.y, nose_world.z, nose_world.visibility],
                            
                           [right_ear_world.x, right_ear_world.y, right_ear_world.z, right_ear_world.visibility],
                           
                           [left_ear_world.x, left_ear_world.y, left_ear_world.z, left_ear_world.visibility],
                            
                           [nose_norm.x, nose_norm.y, nose_norm.z, nose_norm.visibility],
                           
                           [right_ear_norm.x, right_ear_norm.y, right_ear_norm.z, right_ear_norm.visibility],
                          
                           [left_ear_norm.x, left_ear_norm.y, left_ear_norm.z, left_ear_norm.visibility]]
        
        Results.cervical_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_neutral_coord.index = ["neutral_nose_world_coordinates","right_ear_world_coordinates",
                                               "left_ear_world_coordinates","nuetral_nose_norm_coordinates",
                                                "right_ear_norm_coordinates","left_ear_norm_coordinates"]
        print(Results.cervical_neutral_coord)
    else:
        cv2.imwrite("right_cervical.png",image)
        coordinate_list = [[nose_world.x, nose_world.y, nose_world.z, nose_world.visibility],
                            
                           [left_ear_world.x, left_ear_world.y, left_ear_world.z, left_ear_world.visibility],
                            
                           [nose_norm.x, nose_norm.y, nose_norm.z, nose_norm.visibility],
                           
                           [left_ear_norm.x, left_ear_norm.y, left_ear_norm.z, left_ear_norm.visibility]]
    
        Results.cervical_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_right_coord.index = ["right_nose_world_coordinates", "left_ear_world_coordinates",
                                            "right_nose_norm_coordinates","left_ear_norm_coordinates"]
        print(Results.cervical_right_coord)
        
        
    
    
    
    

In [10]:
def mp_pose_flexion(file,side):
    
    mp_holistic = mp.solutions.holistic 
    
    holistic = mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        refine_face_landmarks=True)
    
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    
    draw(annotated_image,results,mp_holistic)
    cv2.imwrite("flexion.png",annotated_image)
    coordinate_list = []
    
    
    if side == -1 or side == 0:
        
        left_shoulder_world = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
        left_shoulder_norm = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
        left_hip_world = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP]
        left_hip_norm = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP]
        
        coordinate_list = [[left_shoulder_world.x,left_shoulder_world.y,
                         left_shoulder_world.z,left_shoulder_world.visibility],
                         
                         [left_shoulder_norm.x,left_shoulder_norm.y,
                         left_shoulder_norm.z,left_shoulder_norm.visibility],
                        
                        [left_hip_world.x,left_hip_world.y,
                         left_hip_world.z,left_hip_world.visibility],
                        
                        [left_hip_norm.x,left_hip_norm.y,
                         left_hip_norm.z,left_hip_norm.visibility]]
        if side == -1:
            Results.flexion_neutral_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_neutral_left_coord.index = Results.flexion_neutral_left_rows
            print(Results.flexion_neutral_left_coord)
        else:
            Results.flexion_extension_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_extension_left_coord.index = Results.flexion_extension_left_rows
            print(Results.flexion_extension_left_coord)
        
    else:
        
        right_shoulder_world = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_shoulder_norm = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_hip_world = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP]
        right_hip_norm = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP]
        
        coordinate_list = [[right_shoulder_world.x, right_shoulder_world.y, 
                            right_shoulder_world.z,right_shoulder_world.visibility],
                         
                         [right_shoulder_norm.x,right_shoulder_norm.y,
                         right_shoulder_norm.z,right_shoulder_norm.visibility],
                        
                        [right_hip_world.x, right_hip_world.y,
                         right_hip_world.z, right_hip_world.visibility],
                        
                        [right_hip_norm.x,right_hip_norm.y,
                         right_hip_norm.z,right_hip_norm.visibility]]
        
        if side == 1:
            Results.flexion_neutral_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_neutral_right_coord.index = Results.flexion_neutral_right_rows
            print(Results.flexion_neutral_right_coord)
        else:
            Results.flexion_extension_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_extension_right_coord.index = Results.flexion_extension_right_rows
            print(Results.flexion_extension_right_coord)
        

# Server Code

In [ ]:
app = Flask(__name__)

@app.route("/")
def showHomePage():
    return "This is home page"

@app.route("/debug",methods=["POST"])
def debug():
    text = request.form["sample"]
    print(text)
    return "received"

####POSTING RELEVANT IMAGES TO SERVER###

@app.route("/tragularLeft",methods=["POST"])
def tragularLeft():
    imageFile = request.files['leftTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/tragularRight",methods=["POST"])
def tragularRight():
    imageFile = request.files['rightTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarLeft",methods=["POST"])
def lumbarLeft():
    imageFile = request.files['leftLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_lumbar_holistic(-1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarNeutral",methods=["POST"])
def lumbarNeutral():
    imageFile = request.files['neutralLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_lumbar_holistic(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarRight",methods=["POST"])
def lumbarRight():
    imageFile = request.files['rightLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_lumbar_holistic(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/intermalleolar",methods=["POST"])
def intermalleolar():
    imageFile = request.files['intermalleolar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_intermalleolar(filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalLeft",methods=["POST"])
def cervicalLeft():
    imageFile = request.files['leftCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,-1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalNeutral",methods=["POST"])
def cervicalNeutral():
    imageFile = request.files['neutralCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,0)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalRight",methods=["POST"])
def cervicalRight():
    imageFile = request.files['rightCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/flexionLeftNeutral",methods=["POST"])
def flexionLeftNeutral():
    imageFile = request.files['leftFlexionNeutral']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,-1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"


@app.route("/flexionLeftExtension",methods=["POST"])
def flexionLeftExtension():
    imageFile = request.files['leftFlexionExtension']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,0)
    print("\nSuccessful upload")
    return "Image uploaded successfully"


@app.route("/flexionRightNeutral",methods=["POST"])
def flexionRightNeutral():
    imageFile = request.files['rightFlexionNeutral']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/flexionRightExtension",methods=["POST"])
def flexionRightExtension():
    imageFile = request.files['rightFlexionExtension']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,2)
    print("\nSuccessful upload")
    return "Image uploaded successfully"
    
    

if __name__ == "__main__":
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://138.38.187.62:5000/ (Press CTRL+C to quit)
138.38.214.43 - - [20/Jul/2022 13:26:35] "POST /debug HTTP/1.1" 200 -


LUMBAR CONNECTED

Received image file name: neutral_lumbar.jpg


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
138.38.214.43 - - [20/Jul/2022 13:26:54] "POST /lumbarNeutral HTTP/1.1" 200 -


                          x         y         z       vis
left_index_world   0.177563  0.093422 -0.125562  0.987595
right_index_world -0.141260  0.060818 -0.226586  0.983652
left_index_hol     0.573249  0.659511 -0.004372  0.000000
right_index_hol    0.377815  0.667323 -0.009098  0.000000
left_wrist         0.602552  0.593775 -0.365544  0.992463
right_wrist        0.357150  0.596285 -0.394553  0.991301
left_elbow         0.596771  0.492499 -0.181857  0.991695
right_elbow        0.377134  0.489215 -0.202681  0.987640

Successful upload


In [68]:
Results.flexion_neutral_right_coord


,x,y,z,vis
right_shoulder_neutral_world,0.034651,-0.477395,-0.238266,0.999997
right_shoulder_neutral_norm,0.443806,0.453936,-0.511103,0.999997
right_hip_neutral_world,0.008334,0.005591,-0.120658,0.999996
right_hip_neutral_norm,0.444173,0.655685,-0.178211,0.999996


In [69]:
Results.intermalleolar_coord

,x,y,z,vis
left_ankle_world,0.689190,0.396863,-0.303811,0.981164
right_ankle_world,-0.528128,0.376756,-0.334474,0.969129
left_ankle_hol,0.783578,0.806050,-0.369591,0.981164
right_ankle_hol,0.210586,0.821205,-0.466344,0.969129
left_knee_hol,0.646230,0.745431,-0.186607,0.997445
right_knee_hol,0.336237,0.747073,-0.314189,0.997143


In [70]:
Results.cervical_left_coord

,x,y,z,vis
left_nose_world_coordinates,0.090276,-0.592641,-0.247032,0.997779
right_ear_world_coordinates,-0.009650,-0.563534,-0.239852,0.998054
left_nose_norm_coordinates,0.090276,-0.592641,-0.247032,0.997779
right_ear_norm_coordinates,0.514177,0.498543,-1.063695,0.998054


In [38]:
def distance(point1,point2):
    return math.sqrt(pow((point1[0]-point2[0]),2)+pow((point1[1]-point2[1]),2))

In [39]:
Results.tragular_left_coord

,x,y,z,vis
left_ear,0.540067,0.405746,-0.268964,0.999900
left_index,0.617129,0.443760,0.001530,0.000000
left_elbow,0.411375,0.514734,-0.324273,0.954215
left_wrist,0.522007,0.429051,-0.061370,0.753456


In [41]:
Results.tragular_left_coord
left_results = Results.tragular_left_coord.values.tolist()
distanceIndexWrist = distance(left_results[1],left_results[3])
distanceIndexWrist
ratioIW = 16/distanceIndexWrist
ratioIW

distanceIndexEar = distance(left_results[1],left_results[0])
print(distanceIndexEar*ratioIW)

14.283775948982672


In [12]:
writer = pd.ExcelWriter('sinead.xlsx',engine='xlsxwriter')
workbook=writer.book

Results.tragular_left_coord.to_excel(writer,sheet_name='Tragular Result',startrow=1 , startcol=0)

Results.tragular_right_coord.to_excel(writer,sheet_name='Tragular Result',startrow=len(Results.tragular_left_coord)+3,startcol=0)

Results.lumbar_neutral_coord.to_excel(writer,sheet_name="Lumbar Result",startrow=1,startcol=0)
Results.lumbar_left_coord.to_excel(writer,sheet_name="Lumbar Result",startrow=10,startcol=0)
Results.lumbar_right_coord.to_excel(writer,sheet_name="Lumbar Result",startrow=19,startcol=0)

Results.intermalleolar_coord.to_excel(writer,sheet_name="Intermalleolar Result",startrow=1,startcol=0)

Results.cervical_neutral_coord.to_excel(writer,sheet_name="Cervical Result",startrow=1,startcol=0)
Results.cervical_left_coord.to_excel(writer,sheet_name="Cervical Result",startrow=8,startcol=0)
Results.cervical_right_coord.to_excel(writer,sheet_name="Cervical Result",startrow=13,startcol=0)
    
#Results.flexion_neutral_left_coord.to_excel(writer,sheet_name="Flexion Result",startrow=1,startcol=0)
#Results.flexion_extension_left_coord.to_excel(writer,sheet_name="Flexion Result",startrow=6,startcol=0)
#Results.flexion_neutral_right_coord.to_excel(writer,sheet_name="Flexion Result",startrow=11,startcol=0)
#Results.flexion_extension_right_coord.to_excel(writer,sheet_name="Flexion Result",startrow=16,startcol=0)

writer.save()




